# mini cluster of ESP32 - test

ref: [Celery Canvas](http://docs.celeryproject.org/en/latest/userguide/canvas.html)

### Imports

In [1]:
import os
import sys
import time

sys.path.append(os.path.abspath(os.path.join('..', '..', '..', 'external', 'private')))
sys.path.append(os.path.abspath(os.path.join('..', '..', 'codes', 'broccoli', 'client')))
sys.path.append(os.path.abspath(os.path.join('..', '..', 'codes', 'broccoli', 'node')))
sys.path.append(os.path.abspath(os.path.join('..', '..', '..', 'external', 'mqtt_network')))

import client
from collections import OrderedDict

from canvas import *
import tasks

My name is Client_366


### Start client
啟動 client 物件。  

在本機上有一個 Broker 物件，負責：
- 管理本機上的 task queue
- 接受使用者發出的運算要求，並將之排入本機上的 task queue
- 通知遠端的 workers 協助處理 tasks
- 將工作發送給 workers 做處理
- 收集 workers 傳回的運算結果
- 將運算結果整合之後，傳回給使用者

而我們透過 client 物件來與 Broker 物件溝通

In [2]:
the_client = client.Client()
the_client.start()

while not the_client.status['Is connected']:            
    time.sleep(1)
    print('Node not ready yet.')


Sending 281 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '2018-04-07 15:49:39.093344 1'), ('kwargs', {'name': 'Client_366'}), ('message_id', '2018-04-07 15:49:39.093344 1'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


[Connected: ('123.240.210.68', 1883)]
[Listen to messages]
Node not ready yet.


### Reset workers
如果需要確保 workers 的狀態都一致，或者需要重新 depoly Python module files 到 workers 上面去，可以發送指令給遠端的 workers，要求做 reboot 回到最初的狀態。

In [3]:
the_client.reset_workers()
time.sleep(15)  # wait until workers ready.


Sending 234 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:49:40.042298 9'), ('message_id', '2018-04-07 15:49:40.042298 9'), ('message_type', 'exec'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('to_exec', 'import machine;machine.reset()')])


Data received: 234 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:49:40.042298 9'), ('message_id', '2018-04-07 15:49:40.042298 9'), ('message_type', 'exec'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('to_exec', 'import machine;machine.reset()')])

No module named 'machine'

Data received: 263 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '5108'), ('kwargs', {'name': 'NodeMCU_b4e62d891371'}), ('message_id', '5108'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Data received: 263 bytes
Message:
OrderedDict([('comm

### Upload tasks module file
Reboot 之後的 workers 只有最基本的功能，如果我們需要 workers 執行額外的功能 (functions)，則需要先將定義這些 functions 的 module 檔案上傳給 workers 並且要求它們 import。  

例如： 我們上傳給每一個 worker 一個`tasks.py`的檔案，其中定義幾個 functions:
```
from canvas import Task

@Task
def add(x, y, op=None):
    return op(x, y) if op else x + y

@Task
def xsum(x):
    return sum(x)

@Task
def mul(x, y, op=None):
    return op(x, y) if op else x * y

@Task
def mapper(word):
    return (word, 1) if len(word) > 3 else None

```

Workers 收到這個`tasks.py`檔案之後，會做`import tasks`的動作，所以就可以呼叫 tasks.add() 這個 function。

In [4]:
tasks_file = os.path.join('..', '..', 'codes', 'broccoli', 'client', 'tasks.py')
the_client.sync_file(tasks_file, load_as_tasks = True)


Sending 550 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:49:55.123497 22'), ('kwargs', {'filename': 'tasks.py', 'file': 'from canvas import Task\n\n@Task\ndef add(x, y, op=None):\n    return op(x, y) if op else x + y\n\n@Task\ndef xsum(x):\n    return sum(x)\n\n@Task\ndef mul(x, y, op=None):\n    return op(x, y) if op else x * y\n\n@Task\ndef mapper(word):\n    return (word, 1) if len(word) > 3 else None\n', 'load_as_tasks': True}), ('message_id', '2018-04-07 15:49:55.123497 22'), ('message_type', 'file'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



<font color='blue'>
## DEMOs

### Chains
Celery [Chains](http://docs.celeryproject.org/en/latest/userguide/canvas.html#chains) 的主要作用是把多個運算**串聯**起來，前一個運算的結果是下一個運算的參數，這樣就可以組成一個完整的運算過程，例如下例中用`chain`組成一個 ((4+4) * 8) * 10  = 640 的計算過程
```
>>> # (4 + 4) * 8 * 10
>>> res = chain(add.s(4, 4), mul.s(8), mul.s(10))
proj.tasks.add(4, 4) | proj.tasks.mul(8) | proj.tasks.mul(10)

>>> res = chain(add.s(4, 4), mul.s(8), mul.s(10))()
>>> res.get()
640
```

我們可以在 ESP32 cluster 上面也做同樣的事情：

In [5]:
ch = chain(tasks.add.s(4, 4), tasks.mul.s(8), tasks.mul.s(10))
ch.get()


Sending 257 bytes

Data received: 550 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:49:55.171399 26'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:49:55.171399 26'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Message:
OrderedDict([('correlation_id', '2018-04-07 15:49:55.123497 22'), ('kwargs', {'filename': 'tasks.py', 'file': 'from canvas import Task\n\n@Task\ndef add(x, y, op=None):\n    return op(x, y) if op else x + y\n\n@Task\ndef xsum(x):\n    return sum(x)\n\n@Task\ndef mul(x, y, op=None):\n    return op(x, y) if op else x * y\n\n@Task\ndef mapper(word):\n    return (word, 1) if len(word) > 3 else None\n', 'load_as_tasks': True}), ('message_id', '2018-04-07 15:49:55.123497 22'), ('message_type', 'file'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 257 bytes
Message:
OrderedDict([('correlation_id', '20


Data received: 526 bytes
Message:
OrderedDict([('correlation_id', '15905'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 64, 'function': 'tasks.mul', 'args': [8, 8], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:49:56.733193 166', 'message_type': 'result', 'task_id': '2018-04-07 15:49:56.733193 166', 'correlation_id': '2018-04-07 15:49:56.733193 166', 'need_result': True, 'sender': 'Client_366'}}), ('message_id', '15905'), ('message_type', 'function'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Sending 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:49:57.918923 260'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:49:57.918923 260'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Data received: 259 bytes

Message:
OrderedDict([('correlation_id', '2018-04-07 15:49:57.91

640

### Groups
Celery [Groups](http://docs.celeryproject.org/en/latest/userguide/canvas.html#groups) 的主要作用是把多個運算**併聯**起來，把很多同質性的運算同時發送給許多遠端的 workers 協助處理，再收集 workers 傳回來的結果彙整成為一個結果集，例如下例中用`group`同時計算 (2+2) 和 (4+4)，結果是 [4, 8]
```
>>> from celery import group
>>> from proj.tasks import add

>>> group(add.s(2, 2), add.s(4, 4))
(proj.tasks.add(2, 2), proj.tasks.add(4, 4))  

>>> g = group(add.s(2, 2), add.s(4, 4))
>>> res = g()
>>> res.get()
[4, 8]
```
我們可以在 ESP32 cluster 上面也做同樣的事情：

In [6]:
gp = group([tasks.add.s(2, 2), tasks.add.s(4, 4)])
gp.get()


Sending 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:49:58.935871 365'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:49:58.935871 365'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:49:58.935871 365'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:49:58.935871 365'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '17316'), ('function', 'dequeue_task'), ('message_id', '17316'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Processed result:
OrderedDict([('correlation_id', '17316'), ('messa

[4, 8]

我們可以用 iterators:
```
>>> group(add.s(i, i) for i in xrange(10))()
```

In [7]:
gp = group([tasks.add.s(i, i) for i in range(10)])
gp.get()


Sending 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:00.407715 536'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:00.407715 536'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:00.407715 536'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:00.407715 536'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '18735'), ('function', 'dequeue_task'), ('message_id', '18735'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890a95'), ('sender', 'NodeMCU_b4e62d890a95')])


Processed result:
OrderedDict([('correlation_id', '18735'), ('messa


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '19603'), ('function', 'dequeue_task'), ('message_id', '19603'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Processed result:
OrderedDict([('correlation_id', '19603'), ('message_id', '2018-04-07 15:50:02.177123 738'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:00.431111 543', 'function': 'tasks.add', 'args': (5, 5), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:00.431111 543', 'task_id': '2018-04-07 15:50:00.431111 543'}, 4)), ('sender', 'Client_366')])


Sending 489 bytes
Message:
OrderedDict([('correlation_id', '19603'), ('message_id', '2018-04-07 15:50:02.177123 738'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e6


Data received: 526 bytes
Message:
OrderedDict([('correlation_id', '22070'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 16, 'function': 'tasks.add', 'args': [8, 8], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:00.431111 546', 'message_type': 'result', 'task_id': '2018-04-07 15:50:00.431111 546', 'correlation_id': '2018-04-07 15:50:00.431111 546', 'need_result': True, 'sender': 'Client_366'}}), ('message_id', '22070'), ('message_type', 'function'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '22173'), ('function', 'dequeue_task'), ('message_id', '22173'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Processed result:
OrderedDict([('correlation_id', '22173'), ('message_id', '2018-04-07 15:50:04.208857 956'), ('messag

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

### Chords
Celery [Chords](http://docs.celeryproject.org/en/latest/userguide/canvas.html#chords) 的主要作用是由兩段運算所組成的，第一段是一個`Groups`運算，其運算的結果會傳給第二段中的運算，作為其運算所需的參數。  

其作用可以用以下的例子來說明，`header`運算的結果會傳給`callback`做進一步的處理：
```
>>> callback = tsum.s()
>>> header = [add.s(i, i) for i in range(10)]
>>> result = chord(header)(callback)
>>> result.get()
90
```
我們可以在 ESP32 cluster 上面也做同樣的事情：

In [8]:
callback = tasks.xsum.s()
header = [tasks.add.s(i, i) for i in range(10)]
async_result = chord(header)(callback)
async_result.get()


Sending 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:04.313167 967'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:04.313167 967'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:04.313167 967'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:04.313167 967'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '22639'), ('function', 'dequeue_task'), ('message_id', '22639'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Processed result:
OrderedDict([('correlation_id', '22639'), ('messa


Data received: 525 bytes
Message:
OrderedDict([('correlation_id', '24384'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 6, 'function': 'tasks.add', 'args': [3, 3], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:04.336227 972', 'message_type': 'result', 'task_id': '2018-04-07 15:50:04.336227 972', 'correlation_id': '2018-04-07 15:50:04.336227 972', 'need_result': True, 'sender': 'Client_366'}}), ('message_id', '24384'), ('message_type', 'function'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Data received: 525 bytes
Message:
OrderedDict([('correlation_id', '24408'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 8, 'function': 'tasks.add', 'args': [4, 4], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:04.336227 973', 'message_type': 'result', 'task_id': '2018-04-07 15:50:04.336227 973', 'correlation_id': '2018-04-07 15:50:04.336227 973', 'need_result': True, 'sen


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '25642'), ('function', 'dequeue_task'), ('message_id', '25642'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Processed result:
OrderedDict([('correlation_id', '25642'), ('message_id', '2018-04-07 15:50:08.018429 1344'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', (None, 0)), ('sender', 'Client_366')])


Sending 209 bytes
Message:
OrderedDict([('correlation_id', '25642'), ('message_id', '2018-04-07 15:50:08.018429 1344'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', (None, 0)), ('sender', 'Client_366')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '25377'), ('function', 'dequeue_task'), ('message_id', '25377'), ('message_type', 'function'), ('need_result', True), ('r

90

上述的運算可以直接寫成：
```
chord(add.s(i, i) for i in xrange(10))(tsum.s()).get()
```
我們可以在 ESP32 cluster 上面也做同樣的事情：

In [9]:
async_result = chord([tasks.add.s(i, i) for i in range(10)])(tasks.xsum.s())
async_result.get()


Sending 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:09.646531 1515'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:09.646531 1515'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Data received: 261 bytes

Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:09.646531 1515'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:09.646531 1515'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '27990'), ('function', 'dequeue_task'), ('message_id', '27990'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890a95'), ('sender', 'NodeMCU_b4e62d890a95')])


Processed result:
OrderedDict([('correlation_id', '27990'), ('m

Message:
OrderedDict([('correlation_id', '28739'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 4, 'function': 'tasks.add', 'args': [2, 2], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:09.669592 1519', 'message_type': 'result', 'task_id': '2018-04-07 15:50:09.669592 1519', 'correlation_id': '2018-04-07 15:50:09.669592 1519', 'need_result': True, 'sender': 'Client_366'}}), ('message_id', '28739'), ('message_type', 'function'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '28843'), ('function', 'dequeue_task'), ('message_id', '28843'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Processed result:
OrderedDict([('correlation_id', '28843'), ('message_id', '2018-04-07 15:50:11.136404 1667'), ('message_type', 'result'), ('r


Processed result:
OrderedDict([('correlation_id', '30169'), ('message_id', '2018-04-07 15:50:12.361476 1803'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:09.669592 1526', 'function': 'tasks.add', 'args': (9, 9), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:09.669592 1526', 'task_id': '2018-04-07 15:50:09.669592 1526'}, 0)), ('sender', 'Client_366')])


Sending 493 bytes

Data received: 529 bytesMessage:
OrderedDict([('correlation_id', '30169'), ('message_id', '2018-04-07 15:50:12.361476 1803'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:09.669592 1526', 'function': 'tasks.add', 'args': (9, 9), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id':


Data received: 561 bytes
Message:
OrderedDict([('correlation_id', '31872'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 90, 'function': 'tasks.xsum', 'args': [[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:12.933953 1857', 'message_type': 'result', 'task_id': '2018-04-07 15:50:12.933953 1857', 'correlation_id': '2018-04-07 15:50:12.933953 1857', 'need_result': True, 'sender': 'Client_366'}}), ('message_id', '31872'), ('message_type', 'function'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])



90

### Map & Starmap
Celery [Map & Starmap](http://docs.celeryproject.org/en/latest/userguide/canvas.html#map-starmap) 的主要作用和 Python 中的`map`指令一樣，會對一個 list 中的每個 element 做指定的運算，例如下例會分別對`range(10)`,`range(100)`做`sum`運算：
```
>>> ~xsum.map([range(10), range(100)])
[45, 4950]
```
我們可以在 ESP32 cluster 上面也做同樣的事情，但是須先使用`list()`對`range`物件做展開：

In [10]:
gp = tasks.xsum.map([list(range(10)), list(range(100))])
gp.get()


Sending 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:13.924441 1966'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:13.924441 1966'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:13.924441 1966'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:13.924441 1966'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '32270'), ('function', 'dequeue_task'), ('message_id', '32270'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890a95'), ('sender', 'NodeMCU_b4e62d890a95')])


Processed result:
OrderedDict([('correlation_id', '32270'), ('m

[45, 4950]

`starmap`的作用和`map`指令一樣，會對一個 list 中的每個 element 做指定的運算，只是會先做 star展開，將一個`list`展開成為 positional arguments：
```
>>> ~add.starmap(zip(range(10), range(10)))
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
```
我們可以在 ESP32 cluster 上面也做同樣的事情，但是須先使用`list()`對`zip`物件做展開：

In [11]:
gp = tasks.add.starmap(list(zip(range(10), range(10))))
gp.get()


Sending 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:14.983101 2058'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:14.983101 2058'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Data received: 261 bytes

Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:14.983101 2058'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:14.983101 2058'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '33356'), ('function', 'dequeue_task'), ('message_id', '33356'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Processed result:
OrderedDict([('correlation_id', '33356'), ('m


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '34072'), ('function', 'dequeue_task'), ('message_id', '34072'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890a95'), ('sender', 'NodeMCU_b4e62d890a95')])


Processed result:
OrderedDict([('correlation_id', '34072'), ('message_id', '2018-04-07 15:50:16.525536 2217'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d890a95'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:15.016229 2065', 'function': 'tasks.add', 'args': (5, 5), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:15.016229 2065', 'task_id': '2018-04-07 15:50:15.016229 2065'}, 4)), ('sender', 'Client_366')])


Sending 493 bytes
Message:
OrderedDict([('correlation_id', '34072'), ('message_id', '2018-04-07 15:50:16.525536 2217'), ('message_type', 'result'), ('receiver', 'NodeMCU


Data received: 529 bytes
Message:
OrderedDict([('correlation_id', '35834'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 14, 'function': 'tasks.add', 'args': [7, 7], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:15.016229 2067', 'message_type': 'result', 'task_id': '2018-04-07 15:50:15.016229 2067', 'correlation_id': '2018-04-07 15:50:15.016229 2067', 'need_result': True, 'sender': 'Client_366'}}), ('message_id', '35834'), ('message_type', 'function'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Data received: 529 bytes
Message:
OrderedDict([('correlation_id', '35859'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 16, 'function': 'tasks.add', 'args': [8, 8], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:15.016229 2068', 'message_type': 'result', 'task_id': '2018-04-07 15:50:15.016229 2068', 'correlation_id': '2018-04-07 15:50:15.016229 2068', 'need_result': Tr

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

### Chunks
Celery [Chunks](http://docs.celeryproject.org/en/latest/userguide/canvas.html#chunks) 的主要作用是把一大串的資料切成指定的份數，分發給遠端的 workers 協處處理，例如：
```
>>> res = add.chunks(zip(range(100), range(100)), 10)()
>>> res.get()
[[0, 2, 4, 6, 8, 10, 12, 14, 16, 18],
 [20, 22, 24, 26, 28, 30, 32, 34, 36, 38],
 [40, 42, 44, 46, 48, 50, 52, 54, 56, 58],
 [60, 62, 64, 66, 68, 70, 72, 74, 76, 78],
 [80, 82, 84, 86, 88, 90, 92, 94, 96, 98],
 [100, 102, 104, 106, 108, 110, 112, 114, 116, 118],
 [120, 122, 124, 126, 128, 130, 132, 134, 136, 138],
 [140, 142, 144, 146, 148, 150, 152, 154, 156, 158],
 [160, 162, 164, 166, 168, 170, 172, 174, 176, 178],
 [180, 182, 184, 186, 188, 190, 192, 194, 196, 198]]
```
我們可以在 ESP32 cluster 上面也做同樣的事情，但是須先使用`list()`對`zip`物件做展開：

In [12]:
ck = tasks.add.chunks(list(zip(range(100), range(100))), 10)
async_result = ck()
async_result.get()


Sending 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:19.905739 2557'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:19.905739 2557'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Data received: 261 bytes

Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:19.905739 2557'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:19.905739 2557'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '38199'), ('function', 'dequeue_task'), ('message_id', '38199'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Processed result:
OrderedDict([('correlation_id', '38199'), ('m


Data received: 528 bytes
Message:
OrderedDict([('correlation_id', '39745'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 8, 'function': 'tasks.add', 'args': [4, 4], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:19.942837 2563', 'message_type': 'result', 'task_id': '2018-04-07 15:50:19.942837 2563', 'correlation_id': '2018-04-07 15:50:19.942837 2563', 'need_result': True, 'sender': 'Client_366'}}), ('message_id', '39745'), ('message_type', 'function'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '39848'), ('function', 'dequeue_task'), ('message_id', '39848'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Processed result:
OrderedDict([('correlation_id', '39848'), ('message_id', '2018-04-07 15:50:22.361483 2903'), ('mes


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '41285'), ('function', 'dequeue_task'), ('message_id', '41285'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Processed result:
OrderedDict([('correlation_id', '41285'), ('message_id', '2018-04-07 15:50:23.601151 3019'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:19.942837 2569', 'function': 'tasks.add', 'args': (10, 10), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:19.942837 2569', 'task_id': '2018-04-07 15:50:19.942837 2569'}, 89)), ('sender', 'Client_366')])


Sending 496 bytes

Data received: 529 bytesMessage:
OrderedDict([('correlation_id', '41285'), ('message_id', '2018-04-07 15:50:23.601151 3019'), ('message_type', 'res

Message:
OrderedDict([('correlation_id', '43053'), ('function', 'dequeue_task'), ('message_id', '43053'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Processed result:
OrderedDict([('correlation_id', '43053'), ('message_id', '2018-04-07 15:50:25.267188 3194'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d890499'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:19.942837 2574', 'function': 'tasks.add', 'args': (15, 15), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:19.942837 2574', 'task_id': '2018-04-07 15:50:19.942837 2574'}, 84)), ('sender', 'Client_366')])


Sending 496 bytes
Message:
OrderedDict([('correlation_id', '43053'), ('message_id', '2018-04-07 15:50:25.267188 3194'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d890499'), ('repl


Data received: 531 bytes
Message:
OrderedDict([('correlation_id', '44701'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 36, 'function': 'tasks.add', 'args': [18, 18], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:19.942837 2577', 'message_type': 'result', 'task_id': '2018-04-07 15:50:19.942837 2577', 'correlation_id': '2018-04-07 15:50:19.942837 2577', 'need_result': True, 'sender': 'Client_366'}}), ('message_id', '44701'), ('message_type', 'function'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '44806'), ('function', 'dequeue_task'), ('message_id', '44806'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Processed result:
OrderedDict([('correlation_id', '44806'), ('message_id', '2018-04-07 15:50:26.886106 3373'), ('

...


Sending 496 bytes
Message:
OrderedDict([('correlation_id', '63676'), ('message_id', '2018-04-07 15:50:46.209147 5193'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d890499'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:19.943872 2645', 'function': 'tasks.add', 'args': (86, 86), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:19.943872 2645', 'task_id': '2018-04-07 15:50:19.943872 2645'}, 13)), ('sender', 'Client_366')])


Data received: 532 bytes
Message:
OrderedDict([('correlation_id', '64332'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 168, 'function': 'tasks.add', 'args': [84, 84], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:19.943872 2643', 'message_type': 'result', 'task_id': '2018-04-07 15:50:19.943872 2643', 'correlation_id': '2018-04-07 15:50:19.943872 2643', 'need_result': True, 'sender': 'Client_366'}}), ('messag


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '65065'), ('function', 'dequeue_task'), ('message_id', '65065'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Processed result:
OrderedDict([('correlation_id', '65065'), ('message_id', '2018-04-07 15:50:47.338030 5299'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:19.943872 2650', 'function': 'tasks.add', 'args': (91, 91), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:19.943872 2650', 'task_id': '2018-04-07 15:50:19.943872 2650'}, 8)), ('sender', 'Client_366')])


Sending 495 bytes
Message:
OrderedDict([('correlation_id', '65065'), ('message_id', '2018-04-07 15:50:47.338030 5299'), ('message_type', 'result'), ('receiver', 'NodeM


Processed result:
OrderedDict([('correlation_id', '66818'), ('message_id', '2018-04-07 15:50:48.766774 5438'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:19.943872 2655', 'function': 'tasks.add', 'args': (96, 96), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:19.943872 2655', 'task_id': '2018-04-07 15:50:19.943872 2655'}, 3)), ('sender', 'Client_366')])


Sending 495 bytes
Message:
OrderedDict([('correlation_id', '66818'), ('message_id', '2018-04-07 15:50:48.766774 5438'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:19.943872 2655', 'function': 'tasks.add', 'args': (96, 96), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:19


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '67730'), ('function', 'dequeue_task'), ('message_id', '67730'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Processed result:
OrderedDict([('correlation_id', '67730'), ('message_id', '2018-04-07 15:50:50.156479 5566'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d890499'), ('reply_to', 'Client_366'), ('result', (None, 0)), ('sender', 'Client_366')])


Sending 209 bytes
Message:
OrderedDict([('correlation_id', '67730'), ('message_id', '2018-04-07 15:50:50.156479 5566'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d890499'), ('reply_to', 'Client_366'), ('result', (None, 0)), ('sender', 'Client_366')])


Data received: 532 bytes
Message:
OrderedDict([('correlation_id', '68046'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': 198, 'function': 'tasks.add', 'args': [99, 99

[[0, 2, 4, 6, 8, 10, 12, 14, 16, 18],
 [20, 22, 24, 26, 28, 30, 32, 34, 36, 38],
 [40, 42, 44, 46, 48, 50, 52, 54, 56, 58],
 [60, 62, 64, 66, 68, 70, 72, 74, 76, 78],
 [80, 82, 84, 86, 88, 90, 92, 94, 96, 98],
 [100, 102, 104, 106, 108, 110, 112, 114, 116, 118],
 [120, 122, 124, 126, 128, 130, 132, 134, 136, 138],
 [140, 142, 144, 146, 148, 150, 152, 154, 156, 158],
 [160, 162, 164, 166, 168, 170, 172, 174, 176, 178],
 [180, 182, 184, 186, 188, 190, 192, 194, 196, 198]]

### Word Count
最後我們以 Hadoop 領域中的 "Hello World" 範例 "Word Count" 來測試。  

我們會把一個文字檔的內容拆解成 words 並將每個 word 發送給 workers 處理，workers 要做的主要是一個`mapper`處理：
```
def mapper(word):
    return (word, 1) if len(word) > 3 else None
```
worker 會將處理的結果傳回來，client 這邊會有一個`reduce`function 將結果彙整。  

我們可以在 ESP32 cluster 上面這樣做：

In [13]:
import word_count

text_file = os.path.join('..', '..', 'codes', 'broccoli', 'client', 'test.txt')
words_count, counts = word_count.count_words(text_file)
print('********** result:\nwords count: {}\n\n{}\n**********'.format(words_count, counts[:10]))


Sending 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:51.663029 5746'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:51.663029 5746'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 261 bytes
Message:
OrderedDict([('correlation_id', '2018-04-07 15:50:51.663029 5746'), ('function', 'fetch_task'), ('kwargs', {'broker': 'Client_366'}), ('message_id', '2018-04-07 15:50:51.663029 5746'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '70073'), ('function', 'dequeue_task'), ('message_id', '70073'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Processed result:
OrderedDict([('correlation_id', '70073'), ('m

Sending 501 bytes
Message:
OrderedDict([('correlation_id', '70857'), ('message_id', '2018-04-07 15:50:53.217531 6006'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d890499'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:51.680082 5751', 'function': 'tasks.mapper', 'args': ('George',), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:51.680082 5751', 'task_id': '2018-04-07 15:50:51.680082 5751'}, 89)), ('sender', 'Client_366')])


Data received: 555 bytes
Message:
OrderedDict([('correlation_id', '70989'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': ['Translated', 1], 'function': 'tasks.mapper', 'args': ['Translated'], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:51.680082 5749', 'message_type': 'result', 'task_id': '2018-04-07 15:50:51.680082 5749', 'correlation_id': '2018-04-07 15:50:51.680082 5749', 'need_result': True, 'sender


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '72730'), ('function', 'dequeue_task'), ('message_id', '72730'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Processed result:
OrderedDict([('correlation_id', '72730'), ('message_id', '2018-04-07 15:50:55.107278 6217'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:51.680082 5756', 'function': 'tasks.mapper', 'args': ('and',), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:51.680082 5756', 'task_id': '2018-04-07 15:50:51.680082 5756'}, 84)), ('sender', 'Client_366')])


Sending 498 bytes
Message:
OrderedDict([('correlation_id', '72730'), ('message_id', '2018-04-07 15:50:55.107278 6217'), ('message_type', 'result'), ('receiver', 'N

Message:
OrderedDict([('correlation_id', '74493'), ('message_id', '2018-04-07 15:50:56.682469 6396'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d890499'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:51.680082 5760', 'function': 'tasks.mapper', 'args': ('meeting',), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:51.680082 5760', 'task_id': '2018-04-07 15:50:51.680082 5760'}, 80)), ('sender', 'Client_366')])


Data received: 543 bytes
Message:
OrderedDict([('correlation_id', '74631'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': ['Lamb', 1], 'function': 'tasks.mapper', 'args': ['Lamb'], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:51.680082 5758', 'message_type': 'result', 'task_id': '2018-04-07 15:50:51.680082 5758', 'correlation_id': '2018-04-07 15:50:51.680082 5758', 'need_result': True, 'sender': 'Client_366'}}), ('message

Message:
OrderedDict([('correlation_id', '76302'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': None, 'function': 'tasks.mapper', 'args': ['a'], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:51.681105 5762', 'message_type': 'result', 'task_id': '2018-04-07 15:50:51.681105 5762', 'correlation_id': '2018-04-07 15:50:51.681105 5762', 'need_result': True, 'sender': 'Client_366'}}), ('message_id', '76302'), ('message_type', 'function'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '76408'), ('function', 'dequeue_task'), ('message_id', '76408'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Processed result:
OrderedDict([('correlation_id', '76408'), ('message_id', '2018-04-07 15:50:58.738079 6631'), ('message_type', 'result')

...


Data received: 543 bytes
Message:
OrderedDict([('correlation_id', '96464'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': ['Wolf', 1], 'function': 'tasks.mapper', 'args': ['Wolf'], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:51.681105 5822', 'message_type': 'result', 'task_id': '2018-04-07 15:50:51.681105 5822', 'correlation_id': '2018-04-07 15:50:51.681105 5822', 'need_result': True, 'sender': 'Client_366'}}), ('message_id', '96464'), ('message_type', 'function'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '96570'), ('function', 'dequeue_task'), ('message_id', '96570'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Processed result:
OrderedDict([('correlation_id', '96570'), ('message_id', '2018-04-07 15:51:18.79188


Data received: 553 bytes
Message:
OrderedDict([('correlation_id', '98152'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': ['pasture"', 1], 'function': 'tasks.mapper', 'args': ['pasture"'], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:51.681105 5827', 'message_type': 'result', 'task_id': '2018-04-07 15:50:51.681105 5827', 'correlation_id': '2018-04-07 15:50:51.681105 5827', 'need_result': True, 'sender': 'Client_366'}}), ('message_id', '98152'), ('message_type', 'function'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '98260'), ('function', 'dequeue_task'), ('message_id', '98260'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890499'), ('sender', 'NodeMCU_b4e62d890499')])


Processed result:
OrderedDict([('correlation_id', '98260'), ('message_id', '2018-04-07 15:51:


Data received: 223 bytes
Message:
OrderedDict([('correlation_id', '99299'), ('function', 'dequeue_task'), ('message_id', '99299'), ('message_type', 'function'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d891371'), ('sender', 'NodeMCU_b4e62d891371')])


Processed result:
OrderedDict([('correlation_id', '99299'), ('message_id', '2018-04-07 15:51:21.738202 9098'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:51.681105 5833', 'function': 'tasks.mapper', 'args': ('the',), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:51.681105 5833', 'task_id': '2018-04-07 15:50:51.681105 5833'}, 7)), ('sender', 'Client_366')])

Data received: 549 bytes

Message:
OrderedDict([('correlation_id', '99648'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': ['grossly', 1], 'funct


Processed result:
OrderedDict([('correlation_id', '100368'), ('message_id', '2018-04-07 15:51:22.954503 9228'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:51.681105 5837', 'function': 'tasks.mapper', 'args': ('not',), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 15:50:51.681105 5837', 'task_id': '2018-04-07 15:50:51.681105 5837'}, 3)), ('sender', 'Client_366')])


Sending 498 bytes
Message:
OrderedDict([('correlation_id', '100368'), ('message_id', '2018-04-07 15:51:22.954503 9228'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', ({'sender': 'Client_366', 'message_type': 'function', 'message_id': '2018-04-07 15:50:51.681105 5837', 'function': 'tasks.mapper', 'args': ('not',), 'need_result': True, 'reply_to': 'Client_366', 'correlation_id': '2018-04-07 



Sending 210 bytes

Data received: 551 bytes
Message:
OrderedDict([('correlation_id', '102656'), ('message_id', '2018-04-07 15:51:24.717025 9397'), ('message_type', 'result'), ('receiver', 'NodeMCU_b4e62d891371'), ('reply_to', 'Client_366'), ('result', (None, 0)), ('sender', 'Client_366')])
Message:
OrderedDict([('correlation_id', '102699'), ('function', 'enqueue_result'), ('kwargs', {'message': {'result': ['grass"', 1], 'function': 'tasks.mapper', 'args': ['grass"'], 'reply_to': 'Client_366', 'message_id': '2018-04-07 15:50:51.681105 5840', 'message_type': 'result', 'task_id': '2018-04-07 15:50:51.681105 5840', 'correlation_id': '2018-04-07 15:50:51.681105 5840', 'need_result': True, 'sender': 'Client_366'}}), ('message_id', '102699'), ('message_type', 'function'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_b4e62d890a95'), ('sender', 'NodeMCU_b4e62d890a95')])


********** result:
words count: 94

[('Lamb', 5), ('grossly', 3), ('Wolf', 2), ('year', 1), ('with', 1), ('voice', 1)

In [14]:
# Stopping
the_client.stop()
the_client = None
print('\n[________________ Demo stopped ________________]\n')

[Closed: ('123.240.210.68', 1883)]

[________________ Demo stopped ________________]

